<a href="https://colab.research.google.com/github/bellottisofia/Innovation_Adoption_Parameters/blob/main/matrice_similarit%C3%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.spatial.distance import pdist, squareform
from sklearn.impute import SimpleImputer

In [3]:
opinion_germany = pd.read_csv('/content/matrix_Germany (4).csv')

In [4]:
opinion_germany.head()

,popolazione,Q11_1,Q11_2,Q11_3,Q11_4,Q11_5,Q11_6,Q11_7,Q11_8,Q11_9,...,Q18_16,Q18_17,Q18_18,Q18_19,Q18_20,Q18_21,Q18_22,Q18_23,Q20,Q21
0,18 t/m 24_Less than 10 km,0.520000,0.400000,0.160000,0.560000,0.280000,0.000000,0.280000,0.400000,0.040000,...,0.370000,0.470000,0.320000,0.510000,0.440000,0.440000,0.450000,0.490000,0.490000,0.400000
1,18 t/m 24_10-19 km,0.354839,0.225806,0.193548,0.548387,0.322581,0.064516,0.322581,0.258065,0.129032,...,0.387097,0.572581,0.314516,0.483871,0.314516,0.435484,0.395161,0.306452,0.379032,0.435484
2,18 t/m 24_20-29 km,0.370370,0.185185,0.148148,0.444444,0.148148,0.037037,0.259259,0.333333,0.185185,...,0.398148,0.425926,0.388889,0.453704,0.416667,0.379630,0.425926,0.361111,0.351852,0.425926
3,18 t/m 24_30-49 km,0.390244,0.219512,0.292683,0.463415,0.219512,0.170732,0.292683,0.365854,0.121951,...,0.347561,0.439024,0.298780,0.414634,0.378049,0.439024,0.371951,0.414634,0.371951,0.402439
4,18 t/m 24_more than 50,0.315789,0.263158,0.421053,0.526316,0.368421,0.210526,0.684211,0.315789,0.105263,...,0.447368,0.368421,0.342105,0.368421,0.407895,0.355263,0.421053,0.421053,0.394737,0.447368


In [5]:


# 1. Separiamo la colonna 'popolazione', salviamo i nomi e manteniamo solo le entrate numeriche
popolazioni = opinion_germany['popolazione'].values
opinioni = opinion_germany.drop(columns=['popolazione']).values


# 4. Calcolo matrice delle distanze
distanze = squareform(pdist(opinioni, metric='euclidean'))

# 5. Calcolo sigma
sigma_media_germany = np.mean(distanze[np.triu_indices_from(distanze, k=1)])
sigma_mediana_germany = np.median(distanze[np.triu_indices_from(distanze, k=1)])

print("Sigma medio:", sigma_media_germany)
print("Sigma mediano:", sigma_mediana_germany)
# 3. Costruzione della matrice di similarità W usando il Gaussian Kernel
W_germany1 = np.exp(-distanze**2 / (2 * sigma_media_germany**2))
W_germany2 = np.exp(-distanze**2 / (2 * sigma_mediana_germany**2))


Sigma medio: 0.8042761411848516
Sigma mediano: 0.7862304925971324


Vogliamo normalizzare le matrici.
Per gestire il fatto che le diagonali siano unitarie, costruiamo la matrice
$W̃ = D^{-1/2}* W*D^{-1/2}$
dove ogni entrata è W_ij divisa per la radice del prodotto dell somme per righe i e j. Questa matrice mantiene la simmetricità


In [6]:

def normalize_symmetric(W):
    # Calcolo della somma per riga (i gradi)
    d = np.sum(W, axis=1)

    # Evita divisioni per zero
    d_inv_sqrt = np.where(d > 0, 1.0 / np.sqrt(d), 0.0)

    # Costruzione della matrice D^{-1/2}
    D_inv_sqrt = np.diag(d_inv_sqrt)

    # Calcolo della matrice normalizzata simmetricamente
    W_tilde = D_inv_sqrt @ W @ D_inv_sqrt

    return W_tilde

In [7]:
normalize_symmetric(W_germany1)
normalize_symmetric(W_germany2)

array([[0.07104012, 0.04162952, 0.04579362, 0.04284421, 0.03178419,
        0.04309109, 0.0469234 , 0.0439796 , 0.0454233 , 0.03820777,
        0.03856847, 0.03708239, 0.04434679, 0.04682301, 0.03777263,
        0.03503107, 0.03296263, 0.02989287, 0.03201718, 0.03633188,
        0.02913246, 0.02840521, 0.03120901, 0.03129673, 0.01981472],
       [0.04162952, 0.07019273, 0.04335329, 0.0434027 , 0.04160756,
        0.04288708, 0.04935568, 0.03981694, 0.04118718, 0.03245797,
        0.03677998, 0.03921116, 0.04227396, 0.04580203, 0.03457676,
        0.04114403, 0.03605659, 0.03069081, 0.03199913, 0.03716528,
        0.03249717, 0.03046252, 0.03303828, 0.03182207, 0.01729736],
       [0.04579362, 0.04335329, 0.08283169, 0.05307184, 0.03995926,
        0.03197898, 0.04246938, 0.03397881, 0.05592178, 0.02779029,
        0.03183174, 0.03080212, 0.03935016, 0.05334425, 0.03956558,
        0.02763111, 0.02921455, 0.02210435, 0.02403115, 0.03372731,
        0.02311798, 0.02436543, 0.02457222, 0.

Questa matrice potrebbe però non essere normalizzata per righe.
Calcoliamo la somma di ogni riga di $W ̃$ e visualizziamo il massimo.

In [8]:
w_max1= W_germany1.sum(axis=1).max()
w_max1

np.float64(17.525710103560467)

In [9]:
w_max2= W_germany2.sum(axis=1).max()
w_max2

np.float64(17.25340569841998)

Dato che il massimo è maggiore di 1, scaliamo la matrice con il fattore
$$
\alpha = \min\left(1, \frac{1}{\tilde{w}_{\max}}\right)
$$
$$
\tilde{W} = \alpha \cdot W
$$


In [10]:
alpha1= min(1.0, 1.0 / w_max1)
alpha1

np.float64(0.0570590289403933)

In [11]:
alpha2= min(1.0, 1.0 / w_max2)
alpha2

np.float64(0.057959571430675705)

In [12]:
W_germany1= alpha1*W_germany1
W_germany2= alpha2*W_germany2

Otteniamo una matrice simmetrica e substocasica

In [13]:
W_germany1.sum(axis=1)

array([0.82288624, 0.83243955, 0.70950135, 0.82188234, 0.70569759,
       0.93595314, 0.99066755, 0.90249747, 0.85179027, 0.88609246,
       0.84049451, 1.        , 0.98759766, 0.90574971, 0.88727035,
       0.90424225, 0.98252475, 0.94028912, 0.9558186 , 0.99670854,
       0.90635665, 0.92027009, 0.95187336, 0.95282914, 0.6263527 ])

In [12]:

matrice_similarità_germany1 = pd.DataFrame(W_germany1, index=popolazioni, columns=popolazioni)
matrice_similarità_germany1.to_csv("similarità_germany_media.csv", index=True)
matrice_similarità_germany2 = pd.DataFrame(W_germany2, index=popolazioni, columns=popolazioni)
matrice_similarità_germany2.to_csv("similarità_germany_mediana.csv", index=True)

Paesi Bassi

In [16]:
opinion_NL = pd.read_csv('/content/matrix_NL (3).csv')

popolazioni = opinion_NL['popolazione'].values
opinioni = opinion_NL.drop(columns=['popolazione']).values


distanze = squareform(pdist(opinioni, metric='euclidean'))


sigma_media_NL = np.mean(distanze[np.triu_indices_from(distanze, k=1)])
sigma_mediana_NL = np.median(distanze[np.triu_indices_from(distanze, k=1)])


print("Sigma medio:", sigma_media_NL)
print("Sigma mediano:", sigma_mediana_NL)

W_NL1 = np.exp(-distanze**2 / (2 * sigma_media_NL**2))
W_NL2 = np.exp(-distanze**2 / (2 * sigma_mediana_NL**2))



Sigma medio: 0.7565074875915685
Sigma mediano: 0.761473945753696


In [17]:
normalize_symmetric(W_NL1)
normalize_symmetric(W_NL2)
w_max1= W_NL1.sum(axis=1).max()
w_max2= W_NL2.sum(axis=1).max()
alpha1= min(1.0, 1.0 / w_max1)
alpha2= min(1.0, 1.0 / w_max2)
W_NL1= alpha1*W_NL1
W_NL2= alpha2*W_NL2

In [18]:
matrice_similarità_NL1 = pd.DataFrame(W_NL1, index=popolazioni, columns=popolazioni)
matrice_similarità_NL1.to_csv("similarità_NL_media.csv", index=True)
matrice_similarità_NL2 = pd.DataFrame(W_NL2, index=popolazioni, columns=popolazioni)
matrice_similarità_NL2.to_csv("similarità_NL_mediana.csv", index=True)

Ungheria

In [19]:
opinion_Hungary = pd.read_csv('/content/matrix_Hungary (3).csv')

popolazioni = opinion_Hungary['popolazione'].values
opinioni = opinion_Hungary.drop(columns=['popolazione']).values

opinioni = np.array(opinioni, dtype=np.float64)


distanze = squareform(pdist(opinioni, metric='euclidean'))


sigma_media_Hungary = np.mean(distanze[np.triu_indices_from(distanze, k=1)])
sigma_mediana_Hungary = np.median(distanze[np.triu_indices_from(distanze, k=1)])


print("Sigma medio:", sigma_media_Hungary)
print("Sigma mediano:", sigma_mediana_Hungary)

W_Hungary1 = np.exp(-distanze**2 / (2 * sigma_media_Hungary**2))
W_Hungary2 = np.exp(-distanze**2 / (2 * sigma_mediana_Hungary**2))

normalize_symmetric(W_Hungary1)
normalize_symmetric(W_Hungary2)
w_max1= W_Hungary1.sum(axis=1).max()
w_max2= W_Hungary2.sum(axis=1).max()
alpha1= min(1.0, 1.0 / w_max1)
alpha2= min(1.0, 1.0 / w_max2)
W_Hungary1= alpha1*W_Hungary1
W_Hungary2= alpha2*W_Hungary2


matrice_similarità_Hungary1 = pd.DataFrame(W_Hungary1, index=popolazioni, columns=popolazioni)
matrice_similarità_Hungary1.to_csv("similarità_Hungary_media.csv", index=True)
matrice_similarità_Hungary2 = pd.DataFrame(W_Hungary2, index=popolazioni, columns=popolazioni)
matrice_similarità_Hungary2.to_csv("similarità_Hungary_mediana.csv", index=True)

Sigma medio: 0.681184134209904
Sigma mediano: 0.6657244952077255


Norvegia

In [20]:
opinion_Norway = pd.read_csv('/content/matrix_Norway (4).csv')

popolazioni = opinion_Norway['popolazione'].values
opinioni = opinion_Norway.drop(columns=['popolazione']).values


opinioni = np.array(opinioni, dtype=np.float64)


distanze = squareform(pdist(opinioni, metric='euclidean'))


sigma_media_Norway = np.mean(distanze[np.triu_indices_from(distanze, k=1)])
sigma_mediana_Norway = np.median(distanze[np.triu_indices_from(distanze, k=1)])


print("Sigma medio:", sigma_media_Norway)
print("Sigma mediano:", sigma_mediana_Norway)

W_Norway1 = np.exp(-distanze**2 / (2 * sigma_media_Norway**2))
W_Norway2 = np.exp(-distanze**2 / (2 * sigma_mediana_Norway**2))

normalize_symmetric(W_Norway1)
normalize_symmetric(W_Norway2)
w_max1= W_Norway1.sum(axis=1).max()
w_max2= W_Norway2.sum(axis=1).max()
alpha1= min(1.0, 1.0 / w_max1)
alpha2= min(1.0, 1.0 / w_max2)
W_Norway1= alpha1*W_Norway1
W_Norway2= alpha2*W_Norway2

matrice_similarità_Norway1 = pd.DataFrame(W_Norway1, index=popolazioni, columns=popolazioni)
matrice_similarità_Norway1.to_csv("similarità_Norway_media.csv", index=True)
matrice_similarità_Norway2 = pd.DataFrame(W_Norway2, index=popolazioni, columns=popolazioni)
matrice_similarità_Norway2.to_csv("similarità_Norway_mediana.csv", index=True)

Sigma medio: 0.8225606577835773
Sigma mediano: 0.8028151577886948


Danimarca

In [22]:
opinion_Denmark = pd.read_csv('/content/matrix_Denmark (3).csv')

popolazioni = opinion_Denmark['popolazione'].values
opinioni = opinion_Denmark.drop(columns=['popolazione']).values

opinioni = np.array(opinioni, dtype=np.float64)


distanze = squareform(pdist(opinioni, metric='euclidean'))


sigma_media_Denmark = np.mean(distanze[np.triu_indices_from(distanze, k=1)])
sigma_mediana_Denmark = np.median(distanze[np.triu_indices_from(distanze, k=1)])


print("Sigma medio:", sigma_media_Denmark)
print("Sigma mediano:", sigma_mediana_Denmark)

W_Denmark1 = np.exp(-distanze**2 / (2 * sigma_media_Denmark**2))
W_Denmark2 = np.exp(-distanze**2 / (2 * sigma_mediana_Denmark**2))


normalize_symmetric(W_Denmark1)
normalize_symmetric(W_Denmark2)
w_max1= W_Denmark1.sum(axis=1).max()
w_max2= W_Denmark2.sum(axis=1).max()
alpha1= min(1.0, 1.0 / w_max1)
alpha2= min(1.0, 1.0 / w_max2)
W_Denmark1= alpha1*W_Denmark1
W_Denmark2= alpha2*W_Denmark2


matrice_similarità_Denmark1 = pd.DataFrame(W_Denmark1, index=popolazioni, columns=popolazioni)
matrice_similarità_Denmark1.to_csv("similarità_Denmark_media.csv", index=True)
matrice_similarità_Denmark2 = pd.DataFrame(W_Denmark2, index=popolazioni, columns=popolazioni)
matrice_similarità_Denmark2.to_csv("similarità_Denmark_mediana.csv", index=True)

Sigma medio: 0.8315012693447297
Sigma mediano: 0.7794788943108493
